<a href="https://colab.research.google.com/github/Jinpeng-Yu/NTU-ML2021-Spring/blob/master/HW01/HW01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>